In [1]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

In [5]:
mnist.train.images.shape , mnist.train.labels.shape

((55000, 784), (55000, 10))

In [28]:
input_width = 28
input_height = 28
input_channel = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
 
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))*n_conv2

In [29]:
input_size_to_hidden

3136

In [31]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channel,n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1" : tf.Variable(tf.random_normal([input_size_to_hidden,n_hidden])),
    "wo" : tf.Variable(tf.random_normal([n_hidden,n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bo" : tf.Variable(tf.random_normal([n_out]))
}

In [32]:
def conv(x,weight,bias,stride=1):
    out = tf.nn.conv2d(x,weight,padding = "SAME",strides = [1,stride,stride,1])
    out = tf.nn.bias_add(out,bias)
    out = tf.nn.relu(out)
    return out

In [36]:
def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize = [1,k,k,1],strides=[1,k,k,1])

In [41]:
def cnn(x , weights , biases):
    x = tf.reshape(x , shape = [-1,input_height,input_width,input_channel])
    conv1 = conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool = maxpooling(conv1 , max_pool1_k)
    
    conv2 = conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool = maxpooling(conv2 , max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool,shape = [-1,input_size_to_hidden])
    hidden_output_before = tf.add(tf.matmul(hidden_input,weights['wh1']),biases['bh1'])
    hidden_output = tf.nn.relu(hidden_output_before)
    
    output = tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    return output

In [42]:
x = tf.placeholder('float',[None,input_pixels])
y = tf.placeholder(tf.int32,[None,n_out])

In [44]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
pred = cnn(x,weights,biases)

In [46]:
predictions = tf.arg_max(pred,1)
true_labels = tf.arg_max(y,1)
correct_pred = tf.equal(predictions,true_labels)

In [ ]:
#pred_val = sess.run(,feed_dict = {x:mnist_test_images , y:})
correct = sess.run(correct_pred,feed_dict={x:mnist.test.images , y:mnist.test.labels})
correct.sum()

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate = 10)
optimize = optimizer.minimize(cost)